# Load data

In [42]:
import numpy as np

In [197]:
import json
import pandas as pd

# Работа с первым датасетом
with open('C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\case_2_data_for_members.json', 'r', encoding='utf-8') as file:
    data_1 = json.load(file, )

data1 = pd.json_normalize(data_1,max_level=2)

education_cr=['Высшее', 'Бакалавр', 'Магистр'] # критерии наличия высшего образования
dataset=[]
for j in range(0, data1.shape[0]):
    data1_conf=pd.json_normalize(data_1[j]['confirmed_resumes'])
    
    data1_conf['answer']=1
    data1_conf['vacancy_uuid']=data_1[j]['vacancy']['uuid']
    data1_conf['vacancy_name']=data_1[j]['vacancy']['name']
    data1_conf['vacancy_description']=data_1[j]['vacancy']['description']
    data1_conf['vacancy_comment']=data_1[j]['vacancy']['comment']
    
    education=[]
    average_workout=[]
    employer=[]
    work_city=[]
    work_position=[]
    work_description=[]
    for i in range(0, data1_conf.shape[0]):
        
        # Выделение датасета с инфой по работе
        try:
            data1_conf_exp=pd.json_normalize(data_1[j]['confirmed_resumes'][i]['experienceItem'])
            data1_conf_exp['starts']=pd.to_datetime(data1_conf_exp['starts'])
            data1_conf_exp['ends']=pd.to_datetime(data1_conf_exp['ends'])
            
            data1_conf_exp['ends'] = data1_conf_exp['ends'].fillna(pd.to_datetime("2024-02-29 00:00:00"))
            
            data1_conf_exp['duration']=(data1_conf_exp['ends']-data1_conf_exp['starts']).dt.days
            closest_date=max(data1_conf_exp['starts'])
            average_workout.append(data1_conf_exp['duration'].mean())
            line=data1_conf_exp.query('starts == @closest_date')
            text=data1_conf_exp['description'].astype(str).sum()
            employer.append(line['employer'].values[0])
            work_city.append(line['city'].values[0])
            work_position.append(line['position'].values[0])
            work_description.append(text)
        except Exception as e:
            print(f'ESCEPTION RASED: {e}')
            print(f'{(j, i)}')
            continue
        
        # Выделение датасета с инфой по учебе
        try:
            par=0
            data1_conf_edu=pd.json_normalize(data_1[j]['confirmed_resumes'][i]['educationItem'])
            for elem in data1_conf_edu['education_level'].unique():
                if elem in education_cr:
                    education.append(1)
                    par=1
                    break
            if par==0:
                education.append(0)
        except Exception as e:
            print(f'ESCEPTION RASED (in edu extraction for confirmed resumes): {e}')
            print(f'{(j, i)}')
            
    data1_conf['education']=pd.Series(education)
    data1_conf['average_workout']=pd.Series(average_workout)
    data1_conf['employer']=pd.Series(employer)
    data1_conf['work_city']=pd.Series(work_city)
    data1_conf['work_position']=pd.Series(work_position)
    data1_conf['work_description']=pd.Series(work_description)
    dataset.append(data1_conf)
    
    # обработка вакансий которые получили отрицательный ответ
    
    data1_fail=pd.json_normalize(data_1[j]['failed_resumes'])
    data1_fail['answer']=0
    data1_fail['vacancy_uuid']=data_1[j]['vacancy']['uuid']
    data1_fail['vacancy_name']=data_1[j]['vacancy']['name']
    data1_fail['vacancy_description']=data_1[j]['vacancy']['description']
    data1_fail['vacancy_comment']=data_1[j]['vacancy']['comment']
    education=[]
    average_workout=[]
    employer=[]
    work_city=[]
    work_position=[]
    work_description=[]
    for i in range(0, data1_fail.shape[0]):
        
        # Выделение датасета с инфой по работе
        try:
            data1_fail_exp=pd.json_normalize(data_1[j]['failed_resumes'][i]['experienceItem'])
            data1_fail_exp['starts']=pd.to_datetime(data1_fail_exp['starts'])
            data1_fail_exp['ends']=pd.to_datetime(data1_fail_exp['ends'])
            
            data1_fail_exp['ends'] = data1_fail_exp['ends'].fillna(pd.to_datetime("2024-02-29 00:00:00"))
            
            data1_fail_exp['duration']=(data1_fail_exp['ends']-data1_fail_exp['starts']).dt.days #.astype('timedelta64[h]')
            closest_date=max(data1_fail_exp['starts'])
            average_workout.append(data1_fail_exp['duration'].mean())
            line=data1_fail_exp.query('starts == @closest_date')
            text=data1_fail_exp['description'].astype(str).sum()
            employer.append(line['employer'].values[0])
            work_city.append(line['city'].values[0])
            work_position.append(line['position'].values[0])
            work_description.append(text)
        except Exception as e:
            print(f'ESCEPTION RASED: {e}')
            print(f'{(j, i)}')
            continue
        
        # Выделение датасета с инфой по учебе
        try:
            par=0
            data1_fail_edu=pd.json_normalize(data_1[j]['failed_resumes'][i]['educationItem'])
            for elem in data1_fail_edu['education_level'].unique():
                if elem in education_cr:
                    education.append(1)
                    par=1
                    break
            if par==0:
                education.append(0)
        except Exception as e:
            print(f'ESCEPTION RASED (in edu extraction for failed resumes): {e}')
            print(f'{(j, i)}')
            continue
            
    data1_fail['education']=pd.Series(education)
    data1_fail['average_workout']=pd.Series(average_workout)
    data1_fail['employer']=pd.Series(employer)
    data1_fail['work_city']=pd.Series(work_city)
    data1_fail['work_position']=pd.Series(work_position)
    data1_fail['work_description']=pd.Series(work_description)
    dataset.append(data1_fail)

dataset = pd.concat(dataset)    

ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(0, 6)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(1, 14)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(2, 7)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(2, 8)
ESCEPTION RASED (in edu extraction for confirmed resumes): 'educationItem'
(3, 5)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(3, 10)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(3, 14)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(3, 20)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(4, 10)
ESCEPTION RASED: 'experienceItem'
(6, 3)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(6, 20)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(6, 27)
ESCEPTION RASED (in edu extraction for failed resumes): 'educationItem'
(6, 33)


In [164]:
dataset[["employer", "work_position", "work_city"]].iloc[:3]

,employer,work_position,work_city
0,ОАО ЭлектроАвтоТяж,java-разработчик,Коломна
1,ЗАО ГаражХмельРыб,Software Java engineer,Люберцы
2,ЗАО ГаражРадиоРыбСбыт,Java-разработчик,Шатура


In [165]:
dataset["work_description"].iloc[0]

' Проекты: Внутренний продукт  а по заказу инструментов разработки для сотрудников; Команда состояла из владельца продукта, аналитика, 4 бэкенд разработчиков, 4 фронтенд разработчиков, 1 тестер. Что делал на проектах: Спроектировал и реализовал новые методы REST API, разрабатывал новый бизнес функционал, исправлял баги; Создавал новые микро-сервисы на SpringBoot, покрыл их тестами и передал пользователям; Покрыл проект юнит-тестами с использованием jUnit, Mockito; Работал в команде, проходил и проводил code-review. Технологии: Java 11, Spring Boot, Hibernate, Spring Data, Spring Security, Maven, Camunda, Postgres, Liquibase, Kafka, jUnit, Mockito, Swagger, Bitbucket  Проекты: Проект по управлению складскими помещениями. Самостоятельно реализовал часть CRM системы для внутреннего и внешнего использования, интеграции с внешними системами, в том числе с таможней, участвовал в реализации функционала на EJB, развертываемым на WildFly; Проект по интеграции call-центра с системой телефонии Av

#### Save baseline dataset

In [198]:
dataset.to_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\base_dataset.csv", index=False)

In [199]:
dataset = pd.read_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\base_dataset.csv")

# Features processing

In [200]:
# drop useless features
cols_to_drop = ["birth_date","first_name", "last_name", "country", "city", "work_city", "experienceItem", "educationItem", "employer"]
dataset = dataset.loc[:, ~dataset.columns.isin(cols_to_drop)]

In [201]:
# use last work position instead of null in about and key skills columns
dataset["about"] = dataset.apply(lambda x: x["work_position"] if x["about"] is None else x["about"], axis=1)
dataset["key_skills"] = dataset.apply(lambda x: x["work_position"] if x["key_skills"] is None else x["key_skills"], axis=1)

In [202]:
# fill null values of answer and education with zeros
dataset["answer"] = dataset["answer"].fillna(0)
dataset["education"] = dataset["education"].fillna(0)

In [203]:
# merge languageItem amd languageItems
dataset["languages"] = dataset.apply(lambda x: 1 if "Английский" in str(x["languageItem"]) or "Английский" in str(x["languageItems"]) else 0, axis=1)
dataset = dataset.drop(["languageItem", "languageItems"], axis=1)

In [204]:
dataset.to_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\df_not_embedding.csv", index=False)

In [205]:
dataset.head()

,uuid,about,key_skills,answer,vacancy_uuid,vacancy_name,vacancy_description,vacancy_comment,education,average_workout,work_position,work_description,languages
0,8c8cf797-2c6b-3f4b-b28b-20d57bd88b82,NaN,"Java, Java Spring Framework, Hibernate, Git, P...",1,779f3a59-206a-3241-adc4-d7db504f960b,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки,1.0,1171.000000,java-разработчик,Проекты: Внутренний продукт а по заказу инст...,0
1,74221d62-5ea6-3a68-8849-25acd97e208b,NaN,"MATLAB, Java, Android SDK, Design Patterns, Ec...",1,779f3a59-206a-3241-adc4-d7db504f960b,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки,1.0,1548.666667,Software Java engineer,Development and testing (unit and integration...,1
2,9d7eae36-11f8-3cac-9cb2-4cb0ff9d0ae7,"Усидчивый и терпеливый программист, обладающи...","Java – 17, Spring Boot 2.7, Liquibase, Spring ...",1,779f3a59-206a-3241-adc4-d7db504f960b,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки,1.0,1197.000000,Java-разработчик,Проект: ДБО в команду ведение счета. Задачи: ...,0
3,da4c44dd-7c00-3f75-98b4-096b533488a4,Занимаюсь разработкой бекенда и фронтенда веб...,"Kotlin, TypeScript, Java, Vue.js, Spring Boot,...",1,779f3a59-206a-3241-adc4-d7db504f960b,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки,1.0,1505.500000,Наставник,Обязанности Разработка и проектирование систе...,0
4,73d59615-b5b2-35fd-a15d-28963fe143d1,Интересны продуктовые проекты с упором на про...,"Java, PostgreSQL, Hibernate, Java Spring Frame...",1,779f3a59-206a-3241-adc4-d7db504f960b,Java разработчик команда Инвестиции,Описание Мы расширяем команды и ищем разработ...,450 на руки,1.0,469.666667,Teamlead,Кредитный калькулятор юридических лиц Профили...,0


In [206]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   uuid                 656 non-null    object 
 1   about                277 non-null    object 
 2   key_skills           603 non-null    object 
 3   answer               656 non-null    int64  
 4   vacancy_uuid         656 non-null    object 
 5   vacancy_name         656 non-null    object 
 6   vacancy_description  656 non-null    object 
 7   vacancy_comment      67 non-null     object 
 8   education            656 non-null    float64
 9   average_workout      655 non-null    float64
 10  work_position        655 non-null    object 
 11  work_description     655 non-null    object 
 12  languages            656 non-null    int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 66.8+ KB


In [223]:
dataset.apply(
    lambda x: (str(x["about"]) + str(x["key_skills"]) + str(x["work_position"]) + str(x["work_description"]))[:1000], axis=1
)

0      nanJava, Java Spring Framework, Hibernate, Git...
1      nanMATLAB, Java, Android SDK, Design Patterns,...
2       Усидчивый и терпеливый программист, обладающи...
3       Занимаюсь разработкой бекенда и фронтенда веб...
4       Интересны продуктовые проекты с упором на про...
                             ...                        
651    nanJavaScript, Vue.js, Vuex, Nuxt.js, REST API...
652    nanJavaScript, TypeScript, Vue.js, Nuxt.js, Vu...
653    Привет! Меня зовут Дамир. Я Frontend разработч...
654    nanJavaScript, TypeScript, Vue, Nuxt.js, React...
655    НАВЫКИ / ОПЫТ • Владение HTML, CSS и JavaScrip...
Length: 656, dtype: object